<a href="https://colab.research.google.com/github/epodkwan/growthfunction/blob/main/gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install flax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 202 kB 5.4 MB/s 
     |████████████████████████████████| 145 kB 49.6 MB/s 
     |████████████████████████████████| 596 kB 44.3 MB/s 
     |████████████████████████████████| 217 kB 50.4 MB/s 
     |████████████████████████████████| 9.1 MB 38.8 MB/s 
     |████████████████████████████████| 51 kB 6.7 MB/s 
     |████████████████████████████████| 72 kB 493 kB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from typing import Sequence
import jax
import optax
import numpy as np
import jax.numpy as jnp
from jax import jit,grad
from flax import linen as nn
from flax.training import train_state,checkpoints

In [4]:
def npy_loader(path):
    return jnp.load(path)

In [5]:
class SimpleMLP(nn.Module):
    features:Sequence[int]

    @nn.compact
    def __call__(self,inputs):
        x=inputs
        for i,feat in enumerate(self.features):
            x=nn.Dense(feat)(x)
            if i != len(self.features)-1:
                x=nn.relu(x)
        return x

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [52]:
layer_sizes=[64,256,256,256]
learning_rate=1e-6
model=SimpleMLP(features=layer_sizes)
temp=jnp.ones(2)
params=model.init(jax.random.PRNGKey(0),temp)
tx=optax.adam(learning_rate=learning_rate,b1=0.99)
opt_state=tx.init(params)
state=train_state.TrainState.create(apply_fn=model.apply,params=params,tx=tx)
restored_state=checkpoints.restore_checkpoint(ckpt_dir="/content/drive/My Drive/Colab Notebooks/checkpoint_0",target=state)

In [59]:
def predict(params,omega_m,h0,j):
    pred=restored_state.apply_fn(params,jnp.array([omega_m,h0]))
    return pred[j]

In [60]:
def gradient_at(i,j):
    omega_m_grad,h0_grad=grad(predict,(1,2))(restored_state.params,cosmo[i,0],cosmo[i,2],j)
    return omega_m_grad,h0_grad

In [61]:
cosmo=npy_loader("/content/drive/My Drive/Colab Notebooks/cosmo.npy")
input_result=npy_loader("/content/drive/My Drive/Colab Notebooks/combined.npy")
z=npy_loader("/content/drive/My Drive/Colab Notebooks/999.npy")[0,:]
omega_m_grad,h0_grad=gradient_at(1,2)
print(omega_m_grad)
print(h0_grad)
drive.flush_and_unmount()

-0.004340131
0.0007578656
